In [2]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from scoring import eval_model
from transformers import pipeline
import os
from scoring import eval_model
from peft import LoraConfig, get_peft_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"
#model_name = "meta-llama/Llama-2-70b-chat-hf"
model_name="mosaicml/mpt-30b"
base_path="res/1002test/"
r=8
train_dataset_path="../database/output/qa_dataset_train_1002.json"
test_dataset_path="../database/output/qa_dataset_test_1002.json"
context_path="../database/output/context0926.json"
do_original_eval=True
full_lora=True

In [4]:

base_path+=f"r_{r}_"
base_path+=f"fullLoRA_{full_lora}_"

In [5]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            quantization_config=bnb_config, 
                                            #device_map="auto",
                                            device_map="cpu",
                                            )


Loading checkpoint shards: 100%|██████████| 7/7 [02:41<00:00, 23.06s/it]


In [ ]:
for name, param in model.named_parameters():
    print(name)

In [5]:
if full_lora:
    target_modules=[
    "embed_tokens",
    "lm_head",
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
    ]
else:
    target_modules=[]

In [6]:
import transformers
from datasets import load_dataset,Dataset
from scoring import generate_prompt
import random
import json

#Q&A
with open(train_dataset_path, "r") as f:
    train_dataset= json.load(f)

with open(test_dataset_path, "r") as f:
    test_dataset= json.load(f)


#context
with open(context_path, 'r') as f:
    context_list = json.load(f)

data_list=[{"text":i} for i in context_list]

q_a_data_list=[{"text": generate_prompt(d)} for d in train_dataset[:]]
data_list.extend(q_a_data_list)
random.shuffle(data_list)

#tokenize
dataset = Dataset.from_dict({"text": [item["text"] for item in data_list]})
tokenizer.pad_token = tokenizer.eos_token
dataset=dataset.map(lambda samples: tokenizer(samples['text']), batched=True)

Map: 100%|██████████| 97053/97053 [00:08<00:00, 11308.13 examples/s]


In [7]:
#eval_model(test_dataset,pipe,f"res/1002test/test.csv")

In [8]:

peft_config = LoraConfig(
        task_type="CAUSAL_LM", inference_mode=False, r=r, lora_alpha=32,
        lora_dropout=0.1,
        target_modules=target_modules,
    )
model = get_peft_model(model, peft_config)

#model.print_trainable_parameters()

train_args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=0,
        num_train_epochs=1,
        learning_rate=2e-5,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
    )

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=train_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

ans_dict={}
epoch=0

pipe= pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=100)
if do_original_eval:
    pass
    #eval_model(test_dataset,pipe,base_path+f"epoch_{epoch}_eval.csv")

for i in range(5):
    training_result=trainer.train()
    loss_dict[i]={"loss":training_result.training_loss    }
    epoch+=1
    eval_model(test_dataset,pipe,base_path+f"epoch_{epoch}_eval.csv")

    with open(base_path+f"loss.json","w") as f:
        json.dump(loss_dict,f)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'Refor

TypeError: Device() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (str type, int index)


In [ ]:
f"{full_lora}"

'True'

In [ ]:
train_dataset

[{'question': 'What are the key interaction properties of phantom materials that need to be well characterized for X-ray imaging applications?',
  'answer': 'The key interaction properties are X-ray absorption and inelastic scattering, according to the text. The phantom materials need to mimic the interaction properties of the tissues or materials they are substituting as closely as possible.',
  'ref_id': 2059,
  'choices': ['The total attenuation of bone tissues is strongly dependent on the bone mineral content. Additionally, the energy dependence varies significantly between soft, spongy bone with bone marrow versus hard, cortical bone due to differences in their chemical composition and average atomic number. The full range of bone densities needs to be considered.',
   'Bone tissue has a wider variation in composition and density, from soft spongy to hard cortical bone. This compositional difference makes the energy dependence more pronounced. Soft tissues have less compositional 